[Neural Ordinary Differential Equations \- MSur](https://msurtsukov.github.io/Neural-ODE/)

In [1]:
import numpy
numpy.linspace(0., 1., 101)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

In [2]:
numpy.diff(numpy.linspace(0., 1., 101))
numpy.cumsum

<function numpy.cumsum(a, axis=None, dtype=None, out=None)>

In [11]:
import numpy
def euler_odesolver(z0, t0, t1, f):
    t_array = numpy.linspace(t0, t1, 100001)
    z = z0
    for t, h in zip(t_array[1:], numpy.diff(t_array)):
        z = z + h*f(z, t)
    return z
euler_odesolver(numpy.array([1,2,3]), 0, 1, lambda x,t : x)

array([2.71826824, 5.43653647, 8.15480471])

In [15]:
import math
math.e * numpy.array([1,2,3])

array([2.71828183, 5.43656366, 8.15484549])

In [10]:
import torch
from torch import nn
from torch import Tensor
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, a):
        """
        Compute f, adf/dz, adf/dp(parameter:theta) & adf/dt
        Args:
            - z:
                - shape:(batch_size, )
            - t:
            - a: adjoint
                - the length of `graph_outputs` should match the length of `outputs`
                    >[Automatic differentiation package \- torch\.autograd — PyTorch master documentation](https://pytorch.org/docs/stable/autograd.html)
        """
        batch_size = z.shape[0]
        
        #calc forward propagation
        out = self.forward(z, t)
        #calc backward propagation
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,),
            (z, t) + tuple(self.parameters()),
            grad_outputs=(a,),
            allow_unused=True,
            retain_graph=True,
        )
        #expand adfdp
        #[adfdps_of_parameter1, adfdps_of_parameter2,...] -> [*adfdp_in_batch1, *adfdp_in_batch2,...]/batchsize
        adfdp = torch.cat([grad_about_a_parameter.flatten() for grad_about_a_parameter in adfdp]).unsqueeze(0)
        adfdp = adfdp.expand(batch_size, -1)#-1 should be the number of parameters
        #expand adfdt
        adfdt = adfdt.expand(batch_size, 1) / batch_size

        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        return torch.cat([p.flatten() for p in self.parameters()])

In [9]:
torch.cat([torch.randn(1,3)]*3,0)

tensor([[0.7207, 1.5835, 0.9731],
        [0.7207, 1.5835, 0.9731],
        [0.7207, 1.5835, 0.9731]])

>We have to separate it from main `torch.nn.Module` because custom backward function can’t be implemented inside Module, but can be implemented inside `torch.autograd.Function`.


In [16]:
class ODEAdjoint(torch.autograd.Function):
    """
    Custom autograd function for neural ode
    """
    @staticmethod
    def forward(context, z0, t, flat_parameters, func):
        """
        underconstruction
        Args:
            context:
            z0:
            t:
            flat_parameters:return of `ODEF.flatten_parameters()`
            func: ODEF
        """
        assert isinstance(func, ODEF)#To avoid emerge type error only in backword()
        
        batch_size, *z_shape = z0.size()
        #???
        with torch.no_grad():
            z = torch.zeros(len(t), batch_size, *z_shape).to(z0)#make same type array & use same device
            for i_t
        #save states for backward
        ctx.func = func
        ctx.save_for_backward(
            t,
            z.clone(),#why take copy?
            flat_parameters,
        )
        
        return z
    
    @staticmethod
    def backward(context, dLdz):
        """
        Args:
            dLdz:
                shape: len(t), batch_size, *z_shape
        """

SyntaxError: invalid syntax (<ipython-input-16-0c15c444c758>, line 22)